## Week two test


1. First, import the primary care physicians dataset (`primary_care_physicians.csv`) into a data frame named `physicians`. 

In [1]:
import pandas as pd

In [2]:
physicians = pd.read_csv('../data/primary_care_physicians.csv')

 2. Filter `physicians` down to just the counties in Tennessee. Save the filtered dataframe back to `physicians`. Verify that the resulting dataframe has 95 rows.

In [3]:
physicians.loc[physicians['state'] == 'Tennessee']

,FIPS,state,county,primary_care_physicians
2432,47001,Tennessee,Anderson,39.0
2433,47003,Tennessee,Bedford,15.0
2434,47005,Tennessee,Benton,3.0
2435,47007,Tennessee,Bledsoe,1.0
2436,47009,Tennessee,Blount,90.0
...,...,...,...,...
2522,47181,Tennessee,Wayne,5.0
2523,47183,Tennessee,Weakley,18.0
2524,47185,Tennessee,White,9.0
2525,47187,Tennessee,Williamson,338.0


In [4]:
physicians = physicians.loc[physicians['state'] == 'Tennessee']
physicians.head(2)

,FIPS,state,county,primary_care_physicians
2432,47001,Tennessee,Anderson,39.0
2433,47003,Tennessee,Bedford,15.0


 3. Look at the distribution of the number of primary care physicians. What do you notice?

4. Now, import the population by county dataset (`population_by_county.csv`) into a DataFrame named `population`.

In [5]:
population = pd.read_csv('../data/population_by_county.csv')

In [6]:
population.head(5)

,FIPS,population,county,state,urban
0,17051,21565,Fayette County,ILLINOIS,Rural
1,17107,29003,Logan County,ILLINOIS,Rural
2,17165,23994,Saline County,ILLINOIS,Rural
3,17097,701473,Lake County,ILLINOIS,Urban
4,17127,14219,Massac County,ILLINOIS,Rural


5. Merge the `physicians` DataFrame with the `population` DataFrame. Keep only the values for Tennessee. When you merge, be sure the include both the `population` and `urban` columns in the merged results. Save the result of the merge back to `physicians`.

In [7]:
physicians['state'] = physicians['state'].str.upper()
physicians.head()

,FIPS,state,county,primary_care_physicians
2432,47001,TENNESSEE,Anderson,39.0
2433,47003,TENNESSEE,Bedford,15.0
2434,47005,TENNESSEE,Benton,3.0
2435,47007,TENNESSEE,Bledsoe,1.0
2436,47009,TENNESSEE,Blount,90.0


In [8]:
physicians = pd.merge(left = population, right = physicians, on = 'FIPS')
physicians.head()


,FIPS,population,county_x,state_x,urban,state_y,county_y,primary_care_physicians
0,47165,183437,Sumner County,TENNESSEE,Urban,TENNESSEE,Sumner,91.0
1,47169,10231,Trousdale County,TENNESSEE,Urban,TENNESSEE,Trousdale,2.0
2,47027,7654,Clay County,TENNESSEE,Rural,TENNESSEE,Clay,2.0
3,47157,936374,Shelby County,TENNESSEE,Urban,TENNESSEE,Shelby,806.0
4,47077,27977,Henderson County,TENNESSEE,Rural,TENNESSEE,Henderson,7.0


6. How many Tennessee counties are considered urban?
Answer:38


In [9]:
 urban_list = physicians.loc[(physicians['urban'] == 'Urban')]
urban_list.shape


(38, 8)

In [10]:
physicians['urban'].value_counts()

Rural    57
Urban    38
Name: urban, dtype: int64

In [11]:
physicians = physicians[['population', 'urban', 'primary_care_physicians']]
physicians.head()

,population,urban,primary_care_physicians
0,183437,Urban,91.0
1,10231,Urban,2.0
2,7654,Rural,2.0
3,936374,Urban,806.0
4,27977,Rural,7.0


How many Tennessee counties are considered urban?
Answer:38

7. The State Health Access Data Assistance Center (SHADAC) (https://www.shadac.org/) classifies counties into three groups based on the number of residents per primary care physician. First, counties with fewer than 1500 residents per primary care physician are considered to have an "adequate" supply. Counties with at least 1500 residents but fewer than 3500 residents per primary care physician are considered to have a "moderately inadequate" supply, and counties with at least 3500 residents per primary care physician are considered to have a "low inadequate" supply. How many counties in Tennessee are in each group?
"adequate" = 14; low inadequate = 31; moderately inadequate=50

In [12]:
adequate = pd.DataFrame(physicians['population'] / physicians['primary_care_physicians'] <= 1500)
adequate.value_counts()


False    81
True     14
dtype: int64

In [19]:
low_inadequate = pd.DataFrame((physicians['population'] / physicians['primary_care_physicians'] >=3500), columns = ['low_inadequate'])
low_inadequate.head()
low_inadequate.value_counts()


low_inadequate
False             64
True              31
dtype: int64

In [22]:
moderately_inadequate = 95-14-31
print(moderately_inadequate)

50


"adequate" = 14
low inadequate = 31
moderately inadequate=50



8.Does there appear to be any detectable relationship between whether a county is urban or rural and its supply of primary care physicians?
no

In [26]:
physicians_rate = pd.DataFrame(physicians['population'] / physicians['primary_care_physicians'], columns = ['ratio'])

physicians_rate.head()

,ratio
0,2015.791209
1,5115.500000
2,3827.000000
3,1161.754342
4,3996.714286


In [27]:
result = pd.concat([physicians, physicians_rate], axis=1)
result.head()

,population,urban,primary_care_physicians,ratio
0,183437,Urban,91.0,2015.791209
1,10231,Urban,2.0,5115.500000
2,7654,Rural,2.0,3827.000000
3,936374,Urban,806.0,1161.754342
4,27977,Rural,7.0,3996.714286


In [37]:
low = result.loc[(result['ratio'] >= 3500) ]
low.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 1 to 93
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   population               31 non-null     int64  
 1   urban                    31 non-null     object 
 2   primary_care_physicians  31 non-null     float64
 3   ratio                    31 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.2+ KB


In [44]:
result[(result['ratio'] <= 1500)]

,population,urban,primary_care_physicians,ratio
3,936374,Urban,806.0,1161.754342
8,687488,Urban,665.0,1033.816541
20,360919,Urban,403.0,895.580645
23,157050,Urban,199.0,789.195980
38,127805,Urban,226.0,565.508850
46,55209,Rural,37.0,1492.135135
56,129927,Urban,90.0,1443.633333
63,97625,Urban,129.0,756.782946
68,225389,Urban,338.0,666.831361
71,59216,Rural,40.0,1480.400000


In [45]:
result[(result['ratio'] >= 3500)]

,population,urban,primary_care_physicians,ratio
1,10231,Urban,2.0,5.115500e+03
2,7654,Rural,2.0,3.827000e+03
4,27977,Rural,7.0,3.996714e+03
9,7962,Rural,2.0,3.981000e+03
11,17623,Rural,5.0,3.524600e+03
13,13344,Rural,0.0,inf
16,12027,Rural,2.0,6.013500e+03
18,61447,Urban,17.0,3.614529e+03
25,32965,Rural,5.0,6.593000e+03
31,25443,Rural,4.0,6.360750e+03


In [46]:
result[(result['ratio'] <= 3500) & (result['ratio'] >= 1500)]


,population,urban,primary_care_physicians,ratio
0,183437,Urban,91.0,2015.791209
5,53679,Urban,21.0,2556.142857
6,14816,Urban,5.0,2963.200000
7,27886,Rural,12.0,2323.833333
10,12104,Rural,6.0,2017.333333
12,200180,Urban,82.0,2441.219512
14,35552,Rural,16.0,2222.000000
15,19847,Rural,9.0,2205.222222
17,30365,Rural,14.0,2168.928571
19,37416,Rural,14.0,2672.571429
